# Evaluation de modèle avec Fiftyone

## Importations et Initialisation

### Librairies

In [ ]:
%load_ext autoreload
%autoreload 1

In [ ]:
from utils import init_notebook, yolo_evaluation, predictions
%aimport datasets, utils.predictions
from utils.predictions import * 

import fiftyone as fo
import sahi
from sahi import AutoDetectionModel
import torch
import pandas as pd
import sys
import os

from pathlib import Path

HOME = Path(os.getcwd()).parents[2]
HOME

print("torch available :",torch.cuda.is_available())
torch.cuda.empty_cache()

### Datasets

In [ ]:
dataset = fo.Dataset.from_dir(
    data_path="../../resources/datasets/coco_datasets/cocass_auvergne_v2/images",  # path to the dataset
    labels_path="../../resources/datasets/coco_datasets/cocass_auvergne_v2/detailed.json",   # path to the labels/annotations file
    dataset_type=fo.types.COCODetectionDataset, # the type of dataset to import
)
dataset

### Lancement de fiftyone

In [ ]:
session = fo.launch_app(dataset, auto=False)
session.open_tab()

## Inférence sur les données pour l'évaluation
Il y a plusieurs manières d'inférer sur les données. Soit en utilisant la méthode SAHI, ce qui est le plus adapté lorsqu'on prend en entrée une feuille entière.<br>
Pour des crops de cartes (nous prenons pour le moment 1200x1200 px) il n'est probablement pas nécessaire de faire des slices des images.

### Instanciation du modèle de détection YOLO

In [ ]:
model_name="yolov10x_yolass_2800multiple_15-patep_imgsz640_detailed"

detection_model = AutoDetectionModel.from_pretrained(
    model_type="yolov8",    #Type du modèle (celui qui a été finetuné)
    model_path="outputs/yolo/yolov10x_yolass_2800multiple_15-patep_imgsz640_detailed4/weights/best.pt",    #Chemin vers le modèle.
    confidence_threshold=0.6,   #Seuil de confiance lors de la détection 
    #Plus le seuil de confiance est élevé, moins il y aura de détections mais plus elles seront précises
    device="cuda:0",  # to use the GPU
)

### Prédiction
Les fonctions crées ci-dessous permettent de faire les prédictions sur les samples données en entrée et de ressortir un objet fiftyone sur lequel fonctionnent les fonctions d'évaluation.

In [ ]:
for sample in dataset.iter_samples(progress=True, autosave=True):
    fo_predict_simple(detection_model,sample, 
        label_field="predictions", 
        overlap_height_ratio=0.6, overlap_width_ratio=0.6)

In [ ]:
session = fo.launch_app(dataset, auto=False)
session.open_tab()